In [ ]:
import earthkit.data

from earthkit.geo import cartography

In [ ]:
countries = ["Ireland"] # List of countries
shapes = cartography.country_polygons(countries, resolution=500e5)

print(len(shapes[0]))

for shape in shapes:
    for point in shape:
        lon = point[1]
        if lon < 0:
            point[1] = lon + 360


In [ ]:
request = {
    "class": "d1",
    "dataset": "on-demand-extremes-dt",
    "stream": "oper",
    "type": "fc",
    "date": "20250926",
    "time": "0000",
    "levtype": "sfc",
    "expver": "0099",
    "param": "167",
    "step": "1/to/9",
    "feature": {
        "type": "polygon",
        "shape": shapes,
    },
}


In [ ]:
data = earthkit.data.from_source("polytope", "destination-earth", request,
                               stream=False,
                               address="polytope-test.lumi.apps.dte.destination-earth.eu",
                               )

In [ ]:
ds = data.to_xarray()

ds

In [ ]:
data_single_step = ds.sel(steps=0, method="nearest")

data_single_step

In [ ]:
import earthkit.plots
chart = earthkit.plots.Map(domain="Ireland")
chart.point_cloud(
    data_single_step['2t'],
    x="longitude",
    y="latitude",
)
chart.title(f"Results from on-demand-extremes-dt")
chart.coastlines()
chart.gridlines()
chart.legend()
chart.show()